In [2]:
import altair as alt
import pandas as pd
import duckdb

In [15]:
energie_df = pd.read_csv('https://dam-api.bfs.admin.ch/hub/api/dam/assets/20764974/master', sep=';')
pop_df = pd.read_csv('https://dam-api.bfs.admin.ch/hub/api/dam/assets/20764979/master', sep=';')

In [27]:
duckdb.query('''
    select *
    from energie_df
    where 1=1
      -- and GEO_NAME in ('Bern', 'Zürich')
      and VARIABLE = 'gaz_t'
    order by VALUE desc nulls last
''').to_df()

,GEO_NR,GEO_NAME,CLASS_HAB,GEOM_PERIOD,VARIABLE,SOURCE,VALUE_PERIOD,UNIT_VALUE,VALUE,STATUS
0,261,Zürich,6,2020-01-01,gaz_t,SVGW,2020,Num,4169908.0,A
1,6621,Genève,6,2020-01-01,gaz_t,SVGW,2020,Num,3038000.0,A
2,2701,Basel,6,2020-01-01,gaz_t,SVGW,2020,Num,2862558.0,A
3,5890,Vevey,3,2020-01-01,gaz_t,SVGW,2020,Num,1518585.0,A
4,5586,Lausanne,6,2020-01-01,gaz_t,SVGW,2020,Num,1432160.0,A
...,...,...,...,...,...,...,...,...,...,...
58,581,Interlaken,1,2020-01-01,gaz_t,SVGW,2020,Num,45060.0,A
59,4436,Romanshorn,2,2020-01-01,gaz_t,SVGW,2020,Num,41205.0,A
60,1301,Einsiedeln,3,2020-01-01,gaz_t,SVGW,2020,Num,33646.0,A
61,177,Pfäffikon,2,2020-01-01,gaz_t,SVGW,2020,Num,27779.0,A


In [30]:
duckdb.query('''
    select e.GEO_NAME, e.VALUE / p.VALUE, e.VALUE, p.VALUE
    from (select * from energie_df) e
    join (select * from pop_df where VARIABLE = 'pop2020' and STATUS = 'A') p on p.GEO_NAME = e.GEO_NAME
    where 1=1
      -- and e.GEO_NAME in ('Bern', 'Zürich')
      and e.VARIABLE = 'gaz_t'
      and e.STATUS = 'A'
    order by 2 desc nulls last
''').to_df()

,GEO_NAME,"(e.""value"") / (p.""value"")",VALUE,VALUE_2
0,Vevey,76.882594,1518585.0,19752.0
1,Solothurn,50.368587,846293.0,16802.0
2,Weinfelden,34.345258,399401.0,11629.0
3,Chiasso,24.743306,187579.0,7581.0
4,Olten,24.677984,456444.0,18496.0
...,...,...,...,...
58,Dietikon,2.807642,78774.0,28057.0
59,Kloten,2.422341,49486.0,20429.0
60,Pfäffikon,2.281830,27779.0,12174.0
61,Einsiedeln,2.070905,33646.0,16247.0


In [3]:
bodennutzung_df = pd.read_csv('https://dam-api.bfs.admin.ch/hub/api/dam/assets/20844688/master', sep=';')

In [134]:
duckdb.query('''
    select distinct
        geo_nr,
        geo_name,
        (select value from bodennutzung_df b2 where b.geo_nr = b2.geo_nr and variable = 'sur_imp_p79') versiegelt79_pct,
        (select value from bodennutzung_df b2 where b.geo_nr = b2.geo_nr and variable = 'sur_imp_p13') versiegelt13_pct,
        (select value from bodennutzung_df b2 where b.geo_nr = b2.geo_nr and variable = 'sur_imp_p13') -
        (select value from bodennutzung_df b2 where b.geo_nr = b2.geo_nr and variable = 'sur_imp_p79') veraenderung
    from bodennutzung_df b
    where 1=1
    and geo_nr ~ '^\d+$'
    and class_hab >= 5
    order by (select value from bodennutzung_df b2 where b.geo_nr = b2.geo_nr and variable = 'sur_imp_p13') - (select value from bodennutzung_df b2 where b.geo_nr = b2.geo_nr and variable = 'sur_imp_p79') desc nulls last
''').to_df()

,GEO_NR,GEO_NAME,versiegelt79_pct,versiegelt13_pct,veraenderung
0,371,Biel/Bienne,26.1,31.6,5.5
1,3203,St. Gallen,20.9,26.0,5.1
2,1061,Luzern,26.8,31.8,5.0
3,5586,Lausanne,22.6,27.1,4.5
4,230,Winterthur,17.6,21.7,4.1
5,261,Zürich,36.7,40.3,3.6
6,351,Bern,25.8,29.3,3.5
7,6621,Genève,62.1,65.5,3.4
8,5192,Lugano,8.4,11.2,2.8
9,2701,Basel,60.0,60.3,0.3


In [140]:
duckdb.query('''
    select
        geo_nr,
        geo_name,
        sum(value),
        (select value from bodennutzung_df b2 where b.geo_nr = b2.geo_nr and variable = 'sur_t7913') agr_pct
    from bodennutzung_df b
    where 1=1
    -- and geo_nr = '351'
    -- and unit_value = 'Per'
    and variable in ('sur_agr13', 'sur_bat13', 'sur_bois13', 'sur_unp13')
    and geo_nr ~ '^\d+$'
    and class_hab >= 4
    group by 1, 2
    order by 3 desc nulls last
''').to_df()

,GEO_NR,GEO_NAME,"sum(""value"")",agr_pct
0,5002,Bellinzona,16411.0,16420.0
1,261,Zürich,8794.0,8793.0
2,5192,Lugano,7598.0,7585.0
3,230,Winterthur,6811.0,6807.0
4,6421,La Chaux-de-Fonds,5573.0,5571.0
5,3901,Chur,5417.0,5424.0
6,351,Bern,5162.0,5162.0
7,355,Köniz,5105.0,5101.0
8,2939,Schaffhausen,4192.0,4185.0
9,5586,Lausanne,4137.0,4138.0


In [152]:
duckdb.query('''
    select distinct
        geo_nr,
        geo_name,
        100 * (select value from bodennutzung_df b2 where b.geo_nr = b2.geo_nr and variable = 'sur_bat13') /
        (select value from bodennutzung_df b2 where b.geo_nr = b2.geo_nr and variable = 'sur_t7913') siedl_pct,
        100 * (select value from bodennutzung_df b2 where b.geo_nr = b2.geo_nr and variable = 'sur_agr13') /
        (select value from bodennutzung_df b2 where b.geo_nr = b2.geo_nr and variable = 'sur_t7913') landw_pct,
        100 * (select value from bodennutzung_df b2 where b.geo_nr = b2.geo_nr and variable = 'sur_bois13') /
        (select value from bodennutzung_df b2 where b.geo_nr = b2.geo_nr and variable = 'sur_t7913') wald_pct,
        100 * (select value from bodennutzung_df b2 where b.geo_nr = b2.geo_nr and variable = 'sur_unp13') /
        (select value from bodennutzung_df b2 where b.geo_nr = b2.geo_nr and variable = 'sur_t7913') unprod_pct,
        (select value from bodennutzung_df b2 where b.geo_nr = b2.geo_nr and variable = 'sur_t7913') total
    from bodennutzung_df b
    where 1=1
    and geo_nr ~ '^\d+$'
    and class_hab >= 5
    order by (select value from bodennutzung_df b2 where b.geo_nr = b2.geo_nr and variable = 'sur_t7913') desc
''').to_df()

,GEO_NR,GEO_NAME,siedl_pct,landw_pct,wald_pct,unprod_pct,total
0,261,Zürich,62.333675,9.507563,26.418742,1.751393,8793.0
1,5192,Lugano,19.261701,10.309822,66.341463,4.258405,7585.0
2,230,Winterthur,34.611429,23.754958,40.605259,1.087116,6807.0
3,351,Bern,46.048043,18.248741,33.339791,2.363425,5162.0
4,5586,Lausanne,45.915901,15.031416,38.811020,0.217496,4138.0
5,3203,St. Gallen,42.051803,27.653631,28.186897,1.929914,3938.0
6,1061,Luzern,48.728522,27.010309,22.027491,2.164948,2910.0
7,2701,Basel,86.708595,3.773585,3.689727,5.953878,2385.0
8,371,Biel/Bienne,47.475224,6.087777,45.398773,1.085418,2119.0
9,6621,Genève,91.964846,1.381042,3.264281,3.201507,1593.0


In [37]:
source = duckdb.query('''
    select b.*,
        case variable
        when 'sur_bat13' then 'Siedlung'
        when 'sur_agr13' then 'Landwirtschaft'
        when 'sur_bois13' then 'Wald'
        when 'sur_unp13' then 'Unproduktiv'
        end var_label,
        (select value from bodennutzung_df b2 where b.geo_nr = b2.geo_nr and variable = 'sur_t7913') total_flaeche
    from bodennutzung_df b
    where 1=1
    and geo_nr ~ '^\d+$'
    and class_hab >= 5
    and variable in ('sur_bat13', 'sur_agr13', 'sur_bois13', 'sur_unp13')
    order by geo_nr::numeric
''').to_df()

alt.Chart(source).mark_bar().encode(
    x=alt.X('VALUE', stack="normalize", title='Anteil', axis=alt.Axis(format='.0%')),
    y=alt.Y('GEO_NAME', title='Stadt'),
    color=alt.Color('var_label', legend=alt.Legend(title='Nutzung')),
    tooltip=[
        alt.Tooltip('VALUE:Q', title='Fläche (ha)'),
        alt.Tooltip('anteil:Q', title='Anteil (%)', format='.2%'),
    ],
).transform_calculate(
    anteil='datum.VALUE / datum.total_flaeche'
).properties(
    title='Bodennutzung Schweizer Städte (50\'000+ Einwohner)'
)

alt.Chart(...)